In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("category_dataset.csv", index_col = False)
df.head()

,Date & Time,Source IP,Destination IP,Source MAC,Destination MAC,Protocol,Flags,Src Port,Dst Port,Length,Class
0,27-02-2020 11:34,192.168.0.50,192.168.0.20,40:f2:e9:9d:44:6b,00:90:e8:5e:d1:66,TCP,AP,60394,502,64,Normal
1,27-02-2020 11:34,192.168.0.20,192.168.0.50,00:90:e8:5e:d1:66,40:f2:e9:9d:44:6b,TCP,A,502,60394,40,Normal
2,27-02-2020 11:34,192.168.0.50,192.168.0.20,40:f2:e9:9d:44:6b,00:90:e8:5e:d1:66,TCP,AP,60394,502,52,Normal
3,27-02-2020 11:34,192.168.0.21,192.168.0.5,20:47:47:87:d4:96,00:1d:9c:a1:89:a4,TCP,S,59170,20000,48,Normal
4,27-02-2020 11:34,192.168.0.21,192.168.0.5,20:47:47:87:d4:96,00:1d:9c:a1:89:a4,TCP,A,59170,20000,40,Normal


In [3]:
def change_class(data):
    lst = ['TCP port & service discovery','Stealth scan OS & version detection','Malware']
    if data in lst:
        return "Malware stealth scan"
    else:
        return data

In [4]:
df["Class"] = df["Class"].apply(change_class)

In [6]:
from joblib import dump, load
enc_bin = load('enc_bin.joblib')
sc_enc = load('sc_enc.joblib')
dtree = load('dtree.joblib')
rfc = load('rfc.joblib')
svm = load('svm.joblib')

In [15]:
from datetime import datetime
start1, start2, start3 = [], [], []
stop1, stop2, stop3 = [], [], []
true_class = []
pred_class1, pred_class2, pred_class3 = [], [], []

In [16]:
for i in range(10000):
    test = df.sample(n=1)
    true_class.append(*list(test["Class"].values))
    test = test[['Source IP', 'Destination IP', 'Source MAC', 'Destination MAC', 'Protocol', 'Flags', 'Src Port', 'Dst Port',
           'Length']].values
    start_time = datetime.now()
    test_bin = enc_bin.transform(test)
    test_scale = sc_enc.transform(test_bin)
    prediction_test = dtree.predict(test_scale)
    stop_time = datetime.now()
    start1.append(start_time)
    stop1.append(stop_time)
    pred_class1.extend(prediction_test)
    
    start_time = datetime.now()
    test_bin = enc_bin.transform(test)
    test_scale = sc_enc.transform(test_bin)
    prediction_test = rfc.predict(test_scale)
    stop_time = datetime.now()
    start2.append(start_time)
    stop2.append(stop_time)
    pred_class2.extend(prediction_test)
    
    start_time = datetime.now()
    test_bin = enc_bin.transform(test)
    test_scale = sc_enc.transform(test_bin)
    prediction_test = svm.predict(test_scale)
    stop_time = datetime.now()
    start3.append(start_time)
    stop3.append(stop_time)
    pred_class3.extend(prediction_test)
    

In [17]:
def find_delay(timestamp):
    start_ms = timestamp[0].microsecond
    stop_ms = timestamp[1].microsecond
    delay = stop_ms - start_ms
    if delay > 0:
        return delay
    else:
        return 1000000+delay

In [18]:
test_dtree_df = pd.DataFrame({"Start time":start1, "Stop time":stop1, "True Class":true_class, "Predicted Class":pred_class1})
test_dtree_df["Delay"] = test_dtree_df[["Start time","Stop time"]].apply(find_delay,axis=1)
test_dtree_df.head()

,Start time,Stop time,True Class,Predicted Class,Delay
0,2020-05-20 18:13:26.757217,2020-05-20 18:13:26.789132,UDP port & service scan,UDP port & service scan,31915
1,2020-05-20 18:13:26.977865,2020-05-20 18:13:27.019995,Malware stealth scan,Malware stealth scan,42130
2,2020-05-20 18:13:27.222703,2020-05-20 18:13:27.255357,MITM,MITM,32654
3,2020-05-20 18:13:27.471681,2020-05-20 18:13:27.506795,Malware stealth scan,Malware stealth scan,35114
4,2020-05-20 18:13:27.751153,2020-05-20 18:13:27.794776,Malware stealth scan,Malware stealth scan,43623


In [19]:
test_rfc_df = pd.DataFrame({"Start time":start2, "Stop time":stop2, "True Class":true_class, "Predicted Class":pred_class2})
test_rfc_df["Delay"] = test_rfc_df[["Start time","Stop time"]].apply(find_delay,axis=1)
test_rfc_df.head()

,Start time,Stop time,True Class,Predicted Class,Delay
0,2020-05-20 18:13:26.789132,2020-05-20 18:13:26.935034,UDP port & service scan,UDP port & service scan,145902
1,2020-05-20 18:13:27.019995,2020-05-20 18:13:27.175810,Malware stealth scan,Malware stealth scan,155815
2,2020-05-20 18:13:27.255357,2020-05-20 18:13:27.419808,MITM,MITM,164451
3,2020-05-20 18:13:27.506795,2020-05-20 18:13:27.683487,Malware stealth scan,Malware stealth scan,176692
4,2020-05-20 18:13:27.794776,2020-05-20 18:13:27.975384,Malware stealth scan,Malware stealth scan,180608


In [20]:
test_svm_df = pd.DataFrame({"Start time":start3, "Stop time":stop3, "True Class":true_class, "Predicted Class":pred_class3})
test_svm_df["Delay"] = test_svm_df[["Start time","Stop time"]].apply(find_delay,axis=1)
test_svm_df.head()

,Start time,Stop time,True Class,Predicted Class,Delay
0,2020-05-20 18:13:26.935034,2020-05-20 18:13:26.977865,UDP port & service scan,UDP port & service scan,42831
1,2020-05-20 18:13:27.175810,2020-05-20 18:13:27.207092,Malware stealth scan,Malware stealth scan,31282
2,2020-05-20 18:13:27.419808,2020-05-20 18:13:27.457119,MITM,MITM,37311
3,2020-05-20 18:13:27.683487,2020-05-20 18:13:27.742118,Malware stealth scan,Malware stealth scan,58631
4,2020-05-20 18:13:27.975384,2020-05-20 18:13:28.025724,Malware stealth scan,Malware stealth scan,50340


In [21]:
test_dtree_df["Delay"].describe()

count     10000.000000
mean      42631.215400
std        8327.920571
min       11870.000000
25%       34114.250000
50%       46405.500000
75%       48240.000000
max      120390.000000
Name: Delay, dtype: float64

In [22]:
test_rfc_df["Delay"].describe()

count     10000.000000
mean     149546.912100
std       14768.470791
min       60383.000000
25%      141947.500000
50%      148034.500000
75%      156859.750000
max      365281.000000
Name: Delay, dtype: float64

In [23]:
test_svm_df["Delay"].describe()

count     10000.00000
mean      43038.20410
std        8344.69859
min        8589.00000
25%       34748.75000
50%       46858.00000
75%       48288.00000
max      135520.00000
Name: Delay, dtype: float64

In [24]:
test_dtree_df.to_csv("dtree_dealy.csv", index=False)
test_rfc_df.to_csv("rfc_dealy.csv", index=False)
test_svm_df.to_csv("svm_dealy.csv", index=False)